In [3]:
"""
Transformation Inspired From:
https://stackoverflow.com/questions/56719138/how-can-i-save-a-librosa-spectrogram-plot-as-a-specific-sized-image/57204349#57204349
"""

# Absolute path of project
project_path = 'C:/Users/arwin/Documents/dev/APS360-PROJECT/' # change this to your machine's

'\nTransformation Inspired From:\nhttps://stackoverflow.com/questions/56719138/how-can-i-save-a-librosa-spectrogram-plot-as-a-specific-sized-image/57204349#57204349\n'

In [4]:
import matplotlib.pyplot as plt
import librosa
import librosa.display
import numpy as np
import pandas as pd
import os
import subprocess
import skimage.io
from skimage.transform import resize
from PIL import Image

Load the first 1.2 seconds of the WAV file

In [3]:
# download_path = project_path + 'data/billboard_audio/' 
download_path = project_path + 'data/random_audio/' 

# df = pd.read_csv( project_path + "data/billboard_2016-2021_song_artist_cleaned_rows.csv")
df = pd.read_csv( project_path + "data/random_2016-2021_song_artist_genres_cleaned_rows.csv")


In [5]:
df.drop_duplicates('song')

,song,artist,genres
0,Most Beautiful,Smooth Lounge Piano,['pianissimo']
1,Precioso Jesus,Gabriel Navarro,['latin talent show']
2,Maggie Valley Bound,Raymond Fairchild,"['banjo', 'traditional bluegrass']"
3,Nighttime Mind Booster Night Time Affirmations,Jason Stephenson,"['432hz', 'guided meditation']"
4,The Company I Keep,Jimmy Donn,['horrorcore']
...,...,...,...
2821,Seabound,Lunchmoney,['lo-fi beats']
2822,Kanto Negro,El Dusty,['nu-cumbia']
2823,Broken,EDF,['chopped and screwed']
2824,Seeking for Me,Mercy's Well,['southern gospel']


Only Run This Cell When Need To Convert All webm Files to MP3

In [6]:

for index, row in enumerate(df.itertuples(), start=1):
    try:
        song = getattr(row, 'song')
        artist = getattr(row, 'artist')

        song_dir_path = download_path + artist + "_" + song

        song_dir_path = song_dir_path + '/' + os.listdir(song_dir_path)[0]
        # print(os.path.splitext(song_dir_path))
        if os.path.exists(song_dir_path):
            print(index, os.listdir( download_path + artist + "_" + song), end=', ')
            command = ['ffmpeg', '-n', '-hwaccel', 'cuda', '-i', song_dir_path, os.path.splitext(song_dir_path)[0] + '.mp3']
            subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
            print(os.listdir( download_path + artist + "_" + song))
        else:
            print("not found")
        
    except Exception as e:
        print(index, e)
        continue
     

1 ['Smooth Lounge Piano_Most Beautiful.webm.mp3', 'Smooth Lounge Piano_Most Beautiful.webm.part'], ['Smooth Lounge Piano_Most Beautiful.webm.mp3', 'Smooth Lounge Piano_Most Beautiful.webm.part']
2 ['Gabriel Navarro_Precioso Jesus.mp3', 'Gabriel Navarro_Precioso Jesus.webm'], ['Gabriel Navarro_Precioso Jesus.mp3', 'Gabriel Navarro_Precioso Jesus.webm']
3 ['Raymond Fairchild_Maggie Valley Bound.mp3', 'Raymond Fairchild_Maggie Valley Bound.webm'], ['Raymond Fairchild_Maggie Valley Bound.mp3', 'Raymond Fairchild_Maggie Valley Bound.webm']
4 ['Jason Stephenson_Nighttime Mind Booster Night Time Affirmations.webm.mp3', 'Jason Stephenson_Nighttime Mind Booster Night Time Affirmations.webm.part'], ['Jason Stephenson_Nighttime Mind Booster Night Time Affirmations.webm.mp3', 'Jason Stephenson_Nighttime Mind Booster Night Time Affirmations.webm.part']
5 ['Jimmy Donn_The Company I Keep.mp3', 'Jimmy Donn_The Company I Keep.webm'], ['Jimmy Donn_The Company I Keep.mp3', 'Jimmy Donn_The Company I Keep.

In [7]:
print("9")

9


In [8]:
def audio_to_spec(audio_path, out_fname="temp.png", sz = (228, 684)):
    
    # 1. CONVERT AUDIO FILE TO SPEC
    song_data, song_sr = librosa.load(os.path.join(dir, audio_path), sr=None)
    # song_data = librosa.feature.melspectrogram(y=song_data, sr=song_sr, n_mels=sz[1], n_fft=1024, hop_length=512, fmax=8000)
    song_data = librosa.feature.melspectrogram(y=song_data, sr=song_sr, n_mels=256, n_fft=2048, hop_length=512*25)

    # 2. Resize for ResNet18 input dims, with height = 228, width = 228*3
    song_data = resize(song_data, sz, order=0, mode='reflect', anti_aliasing=False)

    # 3. Split image into 3 slices by width and insert into each R G B channel
    assert(song_data.shape[1] // 3 == 228)
    channel_sz = song_data.shape[1] // 3
    r, g, b = song_data[:, :channel_sz], song_data[:, channel_sz : 2 * channel_sz], song_data[:, 2 * channel_sz : 3 * channel_sz]
    
    # 4. a - LOG, b - NORM, c - REFLECT, d - INVERT
    def norm_array(X, min=0.0, max=1.0):
        X_std = (X - X.min()) / (X.max() - X.min())
        X_scaled = X_std * (max - min) + min
        return X_scaled
    
    def prepare_array(arr):
        arr = np.log(arr + 1e-9)
        arr = norm_array(arr,0, 255).astype(np.uint8)
        arr = np.flip(arr, axis=0)
        arr = 255 - arr
        return arr

    r = prepare_array(r)
    g = prepare_array(g)
    b = prepare_array(b)
    img = np.stack([r, g, b], axis=-1)


    # 5. Save to img
    skimage.io.imsave(out_fname + ".png", img)


MAIN LOOP

In [9]:
# spec_path = project_path + 'data/billboard_spec/'
spec_path = project_path + 'data/random_spec/'

overwrite_all = False
for i, dir in enumerate(os.listdir(download_path)):
    print(i, dir)
    dir = os.path.join(download_path,dir)
    
    # check if img alr exists
    img_path = os.path.join(spec_path, os.path.basename(dir) + '.png')
    if os.path.exists(img_path) and os.path.getsize(img_path) >= 1024 and not overwrite_all:
        print('exists, skipping...')
        continue
    
    for song_file in os.listdir(dir):
        if song_file.endswith('mp3'):
            try:
                # convert_and_resize_mp3(mp3_path=os.path.join(dir, song_file), img_name=song_file.replace(".mp3", ""))
                audio_to_spec(
                    audio_path=os.path.join(dir, song_file),
                    out_fname= spec_path + song_file.replace(".mp3", "")
                )
            except:
                print("error") 
    


0  '98_Erreur Irrcuprable ()
1 $hawn $kully_True Soldiers
2 $nigg_Dear Emily
3 03 Greedo_Baccstage (feat Shordie Shordie & Wallie The Sensei)
4 03 Greedo_Makin Plays (feat Rich the Kid)
5 03osc_Let Go
6 1 AMVRKA_Raised In The Wild - Acoustic
7 10kCaash_Geek It Up
8 10kCaash_Mulberry Street
9 1600j_Bandz Up
10 18 Kilates_Se Que Te Va a Doler
11 1takejay_URKLE (feat Rucci & Leeky Bandz)
12 1way Frank_Zombie
13 2 Crucial_I Set Trends
14 2 Eleven_Pandemic (feat TF)
15 2 LIVE CREW_How Bout Dem Cowboys Explicit Extended
16 22 Savage_Pay 4 It
17 223Jerm_NA3NA3
18 2shanez_GARLIK ALOE VERA
19 2shanez_JUMP SHIP
20 3 Problems_In & Out
21 38Kea_Heinous Christmas
22 3D Stas_Cat - Virus19xx Remix
23 3xbravo_Look Up to Me
24 48_Happy Xmas operation
25 777villain_Woke Up
26 8-Bit Misfits_Famous
27 83HADES_PERRYTH3PLATYPUS
28 954mari_Teen Rebel
29 A Scent Like Wolves_Bloom (feat JT Cavey)
30 A Sense of Purpose_Mother Night
31 A Step Ahead_Elevated
32 A-Mafia_Me and Gist in the Drop
33 A-Wall_4ever
34 A

C:\Users\arwin\AppData\Local\Temp\ipykernel_36848\3027019505.py:18: RuntimeWarning: invalid value encountered in divide
  X_std = (X - X.min()) / (X.max() - X.min())
C:\Users\arwin\AppData\Local\Temp\ipykernel_36848\3027019505.py:24: RuntimeWarning: invalid value encountered in cast
  arr = norm_array(arr,0, 255).astype(np.uint8)
C:\Users\arwin\AppData\Local\Temp\ipykernel_36848\3027019505.py:36: UserWarning: C:/Users/arwin/Documents/dev/APS360-PROJECT/data/random_spec/Reiki Tribe_Relaxing Soul.png is a low contrast image
  skimage.io.imsave(out_fname + ".png", img)


1838 Rejjie Snow_PURPLE TUESDAY (feat Jesse Boykins III & Joey Bada$$)
1839 Relaxation Meditation Songs Divine_Indian Instrumental II
1840 Relaxation Meditation Songs Divine_Inner Healing Soul (Rain Sounds) II
1841 Relaxation Meditation Songs Divine_Peaceful and Tranquil Music for Relaxation and Baby Massage
1842 Relaxation Meditation Songs Divine_Wildfires Near Me, Hooting Owls in the Night
1843 Relaxing Nature Sounds Collection_Amazonian Wellness
1844 Relaxing Piano Crew_Just Desserts
1845 Relaxing Piano Jazz Music Ensemble_Perfect Day
1846 Relaxing Zen Music Ensemble_Deep Music Therapy
1847 Relaxmydog_Night by the Lake
1848 Remember to Breathe_Thoughts Before Sleep
1849 Remy Ma_Wake Me Up (feat Lil' Kim)
1850 Rene Reed_Jai rv
1851 Renizance_G'z & Hustlaz (Quarantine Song)
1852 resort realism_nitecap
1853 RetroSpecter_Raining Souls
1854 reTYE_Again (From Fruits Basket)
1855 Rhadow_Fragments
1856 Rhett and Link_Christmas Booty (feat Hannah Hart, Grace Helbig & Destorm Power)
1857 Rhya